<center><em>Copyright by Pierian Data Inc.</em></center>
<center><em>For more information, visit us at <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Introduction to Cross Validation

In this lecture series we will do a much deeper dive into various methods of cross-validation. As well as a discussion on the general philosphy behind cross validation. A nice official documentation guide can be found here: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('Data/Advertising.csv')

## Data Example

In [6]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [8]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TV,200.0,147.0425,85.854236,0.7,74.375,149.75,218.825,296.4
radio,200.0,23.2640,14.846809,0.0,9.975,22.90,36.525,49.6
newspaper,200.0,30.5540,21.778621,0.3,12.750,25.75,45.100,114.0
sales,200.0,14.0225,5.217457,1.6,10.375,12.90,17.400,27.0


In [9]:
df.isnull().sum()

TV           0
radio        0
newspaper    0
sales        0
dtype: int64

----
----
----
## Train | Test Split Procedure 

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Test Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Test Data (by creating predictions and comparing to Y_test)
7. Adjust Parameters as Necessary and repeat steps 5 and 6

In [10]:
## CREATE X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [15]:
# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [19]:
# SCALE DATA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) # to avoid data leakage, we don't fit test data.

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Create Model**

In [20]:
from sklearn.linear_model import Ridge

In [21]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha = 100)

In [22]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [23]:
y_predict = model.predict(X_test)

**Evaluation**

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_squared_error(y_test, y_predict)

7.341775789034129

**Adjust Parameters and Re-evaluate**

In [29]:
from sklearn.model_selection import GridSearchCV

In [33]:
parameters = {
    'alpha': [0.001, 0.1, 1, 10, 50, 100]
}

ridge = Ridge()

cv = GridSearchCV(ridge, parameters, cv=5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.1, 1, 10, 50, 100]})

In [34]:
cv.best_params_

{'alpha': 0.1}

In [35]:
model_two = Ridge(alpha=0.1)

In [36]:
model_two.fit(X_train, y_train)

Ridge(alpha=0.1)

In [37]:
y_pred = model_two.predict(X_test)

**Another Evaluation**

In [38]:
mean_squared_error(y_test, y_pred)

2.300652700356322

Much better! We could repeat this until satisfied with performance metrics. (We previously showed RidgeCV can do this for us, but the purpose of this lecture is to generalize the CV process for any model).

----
----
----
## Train | Validation | Test Split Procedure 

This is often also called a "hold-out" set, since you should not adjust parameters based on the final test set, but instead use it *only* for reporting final expected performance.

0. Clean and adjust data as necessary for X and y
1. Split Data in Train/Validation/Test for both X and y
2. Fit/Train Scaler on Training X Data
3. Scale X Eval Data
4. Create Model
5. Fit/Train Model on X Train Data
6. Evaluate Model on X Evaluation Data (by creating predictions and comparing to Y_eval)
7. Adjust Parameters as Necessary and repeat steps 5 and 6
8. Get final metrics on Test set (not allowed to go back and adjust after this!)

In [41]:
import numpy as np
import pandas as pd

In [43]:
df = pd.read_csv('Data/Advertising.csv')

In [44]:
df.head(3)

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3


In [45]:
# Create X and y
X = df.drop('sales', axis=1)
y = df['sales']

In [46]:
######################################################################
#### SPLIT TWICE! Here we create TRAIN | VALIDATION | TEST  #########
####################################################################


from sklearn.model_selection import train_test_split

# 70% of data is training data, set aside other 30%
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Remaining 30% is split into evaluation and test sets
# Each is 15% of the original data size
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [47]:
len(df)

200

In [48]:
len(X_train), len(X_eval), len(X_test)

(140, 30, 30)

In [49]:
# SCALE DATA

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Create Model**

In [68]:
from sklearn.linear_model import Ridge

In [51]:
# Poor Alpha Choice on purpose!
model = Ridge(alpha=100)

In [52]:
model.fit(X_train, y_train)

Ridge(alpha=100)

In [56]:
y_eval_pred = model.predict(X_eval)

**Evaluation**

In [57]:
from sklearn.metrics import mean_squared_error

In [58]:
mean_squared_error(y_eval, y_eval_pred)

7.320101458823871

**Adjust Parameters and Re-evaluate**

In [59]:
from sklearn.model_selection import GridSearchCV

In [71]:
parameters = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

ridge = Ridge()

cv = GridSearchCV(ridge, parameters, cv=5)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10, 100]})

In [72]:
cv.best_params_

{'alpha': 0.1}

In [74]:
y_eval_pred = cv.predict(X_eval)

**Another Evaluation**

In [75]:
mean_squared_error(y_eval, y_eval_pred)

2.359134103282472

**Final Evaluation (Can no longer edit parameters after this!)**

In [76]:
y_test_pred = cv.predict(X_test)

In [77]:
mean_squared_error(y_test_pred, y_test)

2.2421712974301733

----
----
----
## Cross Validation with cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [78]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [79]:
model = Ridge(alpha=100)

In [80]:
from sklearn.model_selection import cross_val_score

In [81]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [82]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [83]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

8.215396464543607

**Adjust model based on metrics**

In [84]:
model = Ridge(alpha=1)

In [85]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [86]:
# Average of the MSE scores (we set back to positive)
abs(scores.mean())

3.3448392965306963

**Final Evaluation (Can no longer edit parameters after this!)**

In [87]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [88]:
y_final_test_pred = model.predict(X_test)

In [89]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514

----
----
----

# Cross Validation with cross_validate

The cross_validate function differs from cross_val_score in two ways:

It allows specifying multiple metrics for evaluation.

It returns a dict containing fit-times, score-times (and optionally training scores as well as fitted estimators) in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be:
        
        - ['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to False by default to save computation time. To evaluate the scores on the training set as well you need to be set to True.

In [93]:
## CREATE X and y
X = df.drop('sales',axis=1)
y = df['sales']

# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [94]:
model = Ridge(alpha=100)

In [95]:
from sklearn.model_selection import cross_validate

In [96]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [97]:
scores

{'fit_time': array([0.00200176, 0.00100136, 0.00100112, 0.00199914, 0.0010016 ]),
 'score_time': array([0.00200129, 0.00200057, 0.00200295, 0.0010004 , 0.00099921]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [98]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.002002,0.002001,-2.312430,-9.325530,-6.449885
1,0.001001,0.002001,-1.746534,-4.944962,-5.589261
2,0.001001,0.002003,-2.562117,-11.396652,-10.339140
3,0.001999,0.001000,-2.018732,-7.024211,-6.619504
4,0.001002,0.000999,-2.279519,-8.385627,-7.755785


In [99]:
pd.DataFrame(scores).mean()

fit_time                        0.001401
score_time                      0.001601
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Adjust model based on metrics**

In [100]:
model = Ridge(alpha=1)

In [101]:
# SCORING OPTIONS:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [102]:
pd.DataFrame(scores).mean()

fit_time                        0.001402
score_time                      0.001601
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Final Evaluation (Can no longer edit parameters after this!)**

In [103]:
# Need to fit the model first!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [104]:
y_final_test_pred = model.predict(X_test)

In [105]:
mean_squared_error(y_test,y_final_test_pred)

2.3190215794287514

----
----